In [5]:
pip install -U scikit-learn

  Attempting uninstall: joblib
    Found existing installation: joblib 1.0.1
    Uninstalling joblib-1.0.1:
      Successfully uninstalled joblib-1.0.1
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.1
    Uninstalling scikit-learn-0.24.1:
      Successfully uninstalled scikit-learn-0.24.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
# import scikit-learn as sk
import numpy as np


try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

time: 0 ns (started: 2025-04-11 10:40:11 +01:00)


In [3]:
#Read in calendar csv

calendar = pd.read_csv('calendar.csv')
calendar.head()

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN
4,241032,2016-01-08,f,NaN


time: 1.02 s (started: 2025-04-11 10:40:12 +01:00)


In [4]:
#Read in listings csv

listings = pd.read_csv('listings.csv')
listings.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",...,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48
2,3308979,https://www.airbnb.com/rooms/3308979,20160104002432,2016-01-04,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,none,Upper Queen Anne is a charming neighborhood fu...,...,10.0,f,NaN,WASHINGTON,f,strict,f,f,2,1.15
3,7421966,https://www.airbnb.com/rooms/7421966,20160104002432,2016-01-04,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,NaN,A charming apartment that sits atop Queen Anne...,none,NaN,...,NaN,f,NaN,WASHINGTON,f,flexible,f,f,1,NaN
4,278830,https://www.airbnb.com/rooms/278830,20160104002432,2016-01-04,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,none,We are in the beautiful neighborhood of Queen ...,...,9.0,f,NaN,WASHINGTON,f,strict,f,f,1,0.89


time: 469 ms (started: 2025-04-11 10:40:29 +01:00)


In [5]:
#Read in reviews csv

reviews = pd.read_csv('reviews.csv')
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb..."
3,7202016,40813543,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...


time: 1.33 s (started: 2025-04-11 10:42:48 +01:00)


In [14]:
# What is the timeframe for this data (max/min dates in calendar)

print(calendar['date'].max())
print(calendar['date'].min())

# Jan 2016 - Jan 2017

2017-01-02
2016-01-04


In [22]:
# How many listings are there in total?
# calendar['listing_id'].count()
print(str(calendar['listing_id'].agg('nunique')) + ' listings in calendar')
print(str(listings['id'].agg('nunique')) + ' listings in listings')
print(str(reviews['listing_id'].agg('nunique')) + ' listings in reviews')

# 3,818 listings

3818 listings in calendar
3818 listings in listings
3191 listings in reviews
time: 78 ms (started: 2023-09-01 17:53:13 +01:00)


In [30]:
# Is the min/max date the same for all listings? Or does it show the dates for when it came to market?

minmaxdate = calendar.groupby('listing_id').agg(max_date = ('date','max'), min_date = ('date','min'))
# minmaxdate = minmaxdate.reset_index()
minmaxdate.head()


,max_date,min_date
listing_id,,
3335,2017-01-02,2016-01-04
4291,2017-01-02,2016-01-04
5682,2017-01-02,2016-01-04
6606,2017-01-02,2016-01-04
7369,2017-01-02,2016-01-04


time: 1.22 s (started: 2023-09-01 18:27:35 +01:00)


In [35]:
minmaxdate[minmaxdate['min_date'] != '2016-01-04'] #returns no rows, so all listings in date range

,max_date,min_date
listing_id,,
3335,2017-01-02,2016-01-04
4291,2017-01-02,2016-01-04
5682,2017-01-02,2016-01-04
6606,2017-01-02,2016-01-04
7369,2017-01-02,2016-01-04
...,...,...
10332096,2017-01-02,2016-01-04
10334184,2017-01-02,2016-01-04
10339144,2017-01-02,2016-01-04


time: 16 ms (started: 2023-09-01 18:28:13 +01:00)


In [36]:
minmaxdate[minmaxdate['max_date'] != '2017-01-02'] #returns no rows, so all listings in date range

,max_date,min_date
listing_id,,


time: 0 ns (started: 2023-09-01 18:28:32 +01:00)
